# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [166]:
# Print scikit-learn version
import sklearn
print('sklearn: %s' % sklearn.__version__)

sklearn: 0.20.2


In [167]:
# import libraries
import pandas as pd
import numpy as np
import os
import pickle
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from scipy.stats import hmean
from scipy.stats.mstats import gmean
import time
import datetime

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\125779\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [142]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('messages_categories',engine)
X = df['message']
Y = df.iloc[:,3:-2]

In [143]:
# Print category columns
category_cols = df.columns[3:-2].tolist()
print(category_cols)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [144]:
# Get stop words in 'English' language
stop_words = stopwords.words("english")

In [145]:
# Print length of stop words in English language
print('Length of stop words in English language is {}'.format(len(stop_words)))

Length of stop words in English language is 179


In [146]:
# Print stop words in English language
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [149]:
# Check if any message contain URL link
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url_count = pd.Series([])
url_count = X.apply(lambda message: len(re.findall(url_regex, message)))
print(type(url_count))
url_count.value_counts().sort_index()    

<class 'pandas.core.series.Series'>


0    25364
1      559
2       78
3       23
4        2
5        1
6        1
Name: message, dtype: int64

From above, we can observe that most of the messages does not have URL links but there are few messages which do contain URL links. There is only 1 observation which contains 5 URL links.

In [150]:
# Define function tokenize to normalize, tokenize and lemmatize text string
def tokenize(text):
    """Normalize, tokenize and lemmatize text string
    
    Args:
    text: string, String containing message for processing
       
    Returns:
    clean_tokens: list, List containing normalized and lemmatized word tokens
    """
    
    # Replace URL links in text string with string 'urlplaceholder'
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Substitute characters in text string which match regular expression r'[^a-zA-Z0-9]'
    # with single whitespace
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Get word tokens from text string
    tokens = word_tokenize(text)
    
    # Instantiate WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # Clean tokens
    clean_tokens = []
    for tok in tokens:
        # convert token to lowercase as stop words are in lowercase
        tok_low = tok.lower() 
        if tok_low not in stop_words:
            # Lemmatize token and remove the leading and trailing spaces from lemmatized token
            clean_tok = lemmatizer.lemmatize(tok_low).lower().strip()
            clean_tokens.append(clean_tok)

    return clean_tokens

In [151]:
# Print first 5 messages and their respective tokens
for idx in X.index.tolist()[0:5]:
    print(X.loc[idx])
    print('-'*100)
    print(tokenize(X.loc[idx]))
    print('*'*100)

Weather update - a cold front from Cuba that could pass over Haiti
----------------------------------------------------------------------------------------------------
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
****************************************************************************************************
Is the Hurricane over or is it not over
----------------------------------------------------------------------------------------------------
['hurricane']
****************************************************************************************************
Looking for someone but no name
----------------------------------------------------------------------------------------------------
['looking', 'someone', 'name']
****************************************************************************************************
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
-------------------------------------

In [152]:
# Print message with url count of 5
print('Index of message with 5 URL links is {}'.format(url_count[url_count == 5].index[0]))
X[url_count[url_count == 5].index[0]]

Index of message with 5 URL links is 12598


'Hurricane Sandy Flight Cancellations: Thousands Of Flights Canceled Due. http://t.co/DMo0tbQE Most read by neighbors in #Roseville #Newarkhappy halloween 2012 (@Frankenstorm Apocalypse - Hurricane Sandy w/ 213 others) http://t.co/DTw9W3kKThe protective cover for the Enterprise failed last night. #NYC #sandy @Space Shuttle Enterprise http://t.co/5jexF6ZG@StormTeam8 @CTPostTrumbull Shelbourne rd Trumbull right next door. Never lost power http://t.co/NqMyZ1H8RT @nytimes: The New York Times is providing free, unlimited access to storm coverage on http://t.co/HkHYUWhW and its mobile apps today.'

In [154]:
# Print message with index location 12598 and its respective tokens
# Index location 12598 contain message with 5 URL links
print(X.loc[12598])
print('-'*100)
print(tokenize(X.loc[12598]))

Hurricane Sandy Flight Cancellations: Thousands Of Flights Canceled Due. http://t.co/DMo0tbQE Most read by neighbors in #Roseville #Newarkhappy halloween 2012 (@Frankenstorm Apocalypse - Hurricane Sandy w/ 213 others) http://t.co/DTw9W3kKThe protective cover for the Enterprise failed last night. #NYC #sandy @Space Shuttle Enterprise http://t.co/5jexF6ZG@StormTeam8 @CTPostTrumbull Shelbourne rd Trumbull right next door. Never lost power http://t.co/NqMyZ1H8RT @nytimes: The New York Times is providing free, unlimited access to storm coverage on http://t.co/HkHYUWhW and its mobile apps today.
----------------------------------------------------------------------------------------------------
['hurricane', 'sandy', 'flight', 'cancellation', 'thousand', 'flight', 'canceled', 'due', 'urlplaceholder', 'read', 'neighbor', 'roseville', 'newarkhappy', 'halloween', '2012', 'frankenstorm', 'apocalypse', 'hurricane', 'sandy', 'w', '213', 'others', 'urlplaceholder', 'protective', 'cover', 'enterpris

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [156]:
# Create a basic pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [171]:
# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
#start_time = time.time()
start_datetime = datetime.datetime.now().replace(microsecond=0)

# Train basic pipeline
pipeline.fit(X_train, Y_train)
#print("--- %s seconds ---" % (time.time() - start_time))
print("--- %s ---" % (datetime.datetime.now().replace(microsecond=0) - start_datetime))

--- 0:01:12 ---


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.